# curran phillips

In [48]:
# import sys
# sys.executable


'/Users/ryantalbot/opt/anaconda3/envs/tf2/bin/python'

In [ ]:
# /Users/ryantalbot/opt/anaconda3/envs/tf2/bin/python -m pip install -U imbalanced-learn

In [9]:
import threadpoolctl
print(threadpoolctl.__version__)


2.2.0


In [47]:
import pandas as pd
import numpy as np
from scipy import stats
from mlxtend.preprocessing import minmax_scaling
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


In [3]:
df = pd.read_csv('../../Combine_Data/men/pb_encoded.csv')
olymp = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_pb.csv')
pbnames = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_pbnames.csv')
pbolymp = pd.read_csv('../../Data/cleandata22-23/men22_23.csv')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         506 non-null    int64  
 1   D            506 non-null    float64
 2   E            506 non-null    float64
 3   ND           506 non-null    float64
 4   Total        506 non-null    float64
 5   year         506 non-null    int64  
 6   medal        506 non-null    int64  
 7   Name         506 non-null    int64  
 8   Nation       506 non-null    int64  
 9   round_final  506 non-null    bool   
 10  round_qual   506 non-null    bool   
dtypes: bool(2), float64(4), int64(5)
memory usage: 36.7 KB


In [5]:
y = df['medal']
X = df.drop(columns=['medal'])

# Split the data into training and testing sets
# CHANGED RANDOM STATE TO HAVE A LARGER NUMBER OF VALUES IN TEST DF
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=19)

# Initialize classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "SVM": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Network": MLPClassifier(),
}

results = {}

# Iterate through each classifier
for name, clf in classifiers.items():
    # Create a pipeline with StandardScaler for classifiers that require it
    if name in ["SVM", "K-Nearest Neighbors", "Neural Network"]:
        clf = make_pipeline(StandardScaler(), clf)

    # Fit the model to the training data
    clf.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = clf.predict(X_test)

    # Evaluate the model's performance
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    results[name] = {
        "accuracy": accuracy,
        "classification_report": report,
        "confusion_matrix": conf_matrix,
    }

# Print the results
for name, result in results.items():
    print(f"Classifier: {name}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"Classification Report:\n{result['classification_report']}")
    print(f"Confusion Matrix:\n{result['confusion_matrix']}\n")


/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

Classifier: Random Forest
Accuracy: 0.9608
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98        96
           1       0.75      0.50      0.60         6

    accuracy                           0.96       102
   macro avg       0.86      0.74      0.79       102
weighted avg       0.96      0.96      0.96       102

Confusion Matrix:
[[95  1]
 [ 3  3]]

Classifier: AdaBoost
Accuracy: 0.9902
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        96
           1       1.00      0.83      0.91         6

    accuracy                           0.99       102
   macro avg       0.99      0.92      0.95       102
weighted avg       0.99      0.99      0.99       102

Confusion Matrix:
[[96  0]
 [ 1  5]]

Classifier: SVM
Accuracy: 0.9412
Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.0

/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-

Classifier: AdaBoost
Accuracy: 0.9902

In [6]:
X_train.head()

,Rank,D,E,ND,Total,year,Name,Nation,round_final,round_qual
84,25,6.2,8.266,0.0,14.466,2018,141,39,False,True
257,1,6.3,8.733,0.0,15.033,2019,166,76,False,True
417,161,4.9,7.233,0.0,12.133,2019,64,2,False,True
461,5,6.0,7.966,0.0,13.966,2019,189,4,False,True
392,136,4.7,8.066,0.0,12.766,2019,149,30,False,True


In [7]:
olymp.head()


,Rank,D Score,E Score,Pen.,Total,year,Name,nation,round_AAfinal,round_TeamFinal,round_TeamQual,round_final,round_qual
0,23.0,3.9,6.900,0.0,10.800,2022,236,61,False,False,False,False,True
1,23.0,3.9,6.900,0.0,10.800,2022,236,61,False,False,False,False,True
2,54.0,4.4,8.066,0.0,12.466,2023,236,61,False,False,False,False,True
3,12.0,5.8,8.400,0.0,14.200,2022,531,26,True,False,False,False,False
4,26.0,5.8,8.500,0.0,14.300,2022,531,26,False,False,False,False,True


In [12]:
class_weight

32.733333333333334

In [72]:
# Use RandomUnderSampler to under-sample the majority class
sampler = RandomUnderSampler(sampling_strategy='all', random_state=233)
X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)

# Initialize the AdaBoost classifier with the custom base estimator
clf = AdaBoostClassifier()

# Fit the model to the resampled training data
clf.fit(X_resampled, y_resampled)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the results
print("AdaBoost Classifier with a Custom Base Estimator (RandomForest)")
print(f"Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n{report}")
print(f"Confusion Matrix:\n{conf_matrix}")

AdaBoost Classifier with a Custom Base Estimator (RandomForest)
Accuracy: 0.8268
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.82      0.90       123
           1       0.15      1.00      0.27         4

    accuracy                           0.83       127
   macro avg       0.58      0.91      0.58       127
weighted avg       0.97      0.83      0.88       127

Confusion Matrix:
[[101  22]
 [  0   4]]


In [65]:
olymp = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_pb.csv')


In [66]:
olymp.rename(columns={'D Score': 'D', 'E Score': 'E',
                     'Pen.': 'ND', 'nation': 'Nation'}, inplace=True)

In [67]:
olymp=olymp.drop(columns=['round_TeamFinal', 'round_AAfinal', 'round_TeamQual'])

In [68]:
y_pred = clf.predict(olymp)


In [69]:
ypreds  = pd.Series(y_pred)
olymp['ypred']=ypreds


In [70]:
olymp['ypred'].value_counts()

ypred
0    1494
1     880
Name: count, dtype: int64

In [57]:
filtered_df = olymp[olymp['ypred'] == 1]

In [58]:
filtered_df['Name'].unique()

array([748, 594, 626, 513, 520, 740, 318, 337,  80, 222, 495,  10,   3,
       242, 231, 413, 197, 536, 238, 487, 102, 104,  92, 324, 420, 101,
       273, 267, 240, 606, 367, 368, 256, 499, 257, 529, 395, 299,  94,
        65, 278, 679, 450,  63, 650, 687, 634, 199, 292, 547, 303, 110,
       722, 504, 700, 338,  91, 597, 285, 467, 714, 144, 311, 515, 509,
       698, 142, 505, 662,  66, 573, 508,  57, 736, 386, 545, 307, 497,
       741, 168, 166, 308, 624, 579, 754,  58,  86, 582, 365, 361, 661,
       390, 448, 759, 516, 493, 674, 443, 425, 506, 286, 295, 690, 732,
       275,  50, 120, 150, 481, 691, 114, 604, 103, 466, 409,  93,  67,
       389, 206, 524, 359, 414, 540, 747, 746, 511,  41,  42, 628, 358,
       659,  20, 666, 585,  15, 462, 475, 139, 138, 342, 564, 250, 719,
       233, 235, 234, 123, 600, 158, 392, 671, 527, 223, 552, 551, 673,
       135,  97,  90, 383,  83, 718, 658, 394, 384, 716,  55, 296, 252,
       670, 449, 652, 583, 555, 218, 192, 133, 485, 486, 265, 65

In [24]:
name_encoded_values = [145,  94,  74]

# Filter the DataFrame to get the corresponding "Name" values
result = pbnames.loc[pbnames['Name_encoded'].isin(name_encoded_values), 'Name']


In [25]:
result.unique()

array(['daniel alarcon', 'caden clinton', 'austin padgett'], dtype=object)

In [26]:
pbolymp["Name"]=pbolymp["Name"].str.lower()

In [27]:
names_to_filter = ['ferhat arican', 'curran phillips', 'carlos edriel yulo']

# Filter the DataFrame to get rows with the specified "Name" values
result = pbolymp[pbolymp['Name'].isin(names_to_filter)]


In [28]:
result['Country'].value_counts()

Country
TUR    63
PHI    49
USA    29
Name: count, dtype: int64

In [29]:
result[result['Country']=='USA']

,Name,Country,Date,Gender,Competition,Round,Location,Apparatus,Rank,D_Score,E_Score,Penalty,Score,Total_Score
10836,curran phillips,USA,NaN,m,2022 Koper World Challenge Cup,final,"Koper, Slovenia",PB,7.0,6.3,6.650,0.0,12.950,12.950
10837,curran phillips,USA,NaN,m,2022 Koper World Challenge Cup,qual,"Koper, Slovenia",PB,1.0,6.7,8.050,0.0,14.750,14.750
10838,curran phillips,USA,2022-08-20,m,2022 U.S. Championships,qual,"Tampa, FL",HB,0.0,5.5,6.450,0.0,11.950,11.950
10839,curran phillips,USA,2022-08-20,m,2022 U.S. Championships,final,"Tampa, FL",HB,0.0,5.7,7.550,0.0,13.250,13.250
10840,curran phillips,USA,2022-08-20,m,2022 U.S. Championships,qual,"Tampa, FL",PB,0.0,6.9,8.450,0.0,15.350,15.350
10841,curran phillips,USA,2022-08-20,m,2022 U.S. Championships,final,"Tampa, FL",PB,0.0,6.9,8.400,0.0,15.300,15.300
10842,curran phillips,USA,2022-08-20,m,2022 U.S. Championships,qual,"Tampa, FL",VT,0.0,5.2,9.200,0.0,14.400,14.400
10843,curran phillips,USA,2022-08-20,m,2022 U.S. Championships,final,"Tampa, FL",VT,0.0,5.2,8.850,0.0,14.050,14.050
10844,curran phillips,USA,2022-02-27,m,2022 Winter Cup,AAfinal,"Frisco, Texas",HB,1.0,5.7,7.950,0.0,13.650,13.650
10845,curran phillips,USA,2022-02-27,m,2022 Winter Cup,AAfinal,"Frisco, Texas",PB,1.0,6.7,8.450,0.0,15.150,15.150
